##### Analysis results using radiation processor- radiation_atmos_av_mon
The radiation processor makes plots to compare model and observed radiation field variables for the available observation sources specified in the processor.  The plots contain three panels to display "model", "observed", and "model minus observed" two-dimensional latitude/longitude climatological field variables.

Original author: Charles Seman, GFDL. 

To use this notebook, please ensure the Input parameters are set and run it in your jupyter notebook environment with all the dependencies installed from this git repository. Input datasets needed are available at:
https://www.gfdl.noaa.gov/am4.0-model
 

Note: 
This notebook was run at GFDL in a Linux environment. The environment variables used along with software requirements is documented below for your reference.

Please note that the data used by any script in this analysis suite can be found in the git repo to be able to reproduce the analysis in your local notebook. Any CMIP data references can be found in GFDL's Data Portal. 
Ref. https://www.gfdl.noaa.gov/am4.0-model


in your env Flexible Runtime Environment. The following environment variables will allow a user to run this analysis script contained within the git repo without explicit "module load". But, the user is expected to copy the directories referenced in the following from the git repo. Also, we will set up an alias to gcp commands. GCP is a GFDL home-grown smart data transfer utility. For public facing scripts, it is enough that the user uses "cp" or other copy tools relevant to their runtime environment.

Acknowledgment: 
GFDL Modeling Services and Data Portal Team.
Charles Seman, Huan Guo et al. 


Software Requirements

1. NETCDF 4.2
2. HDF 1.8.8
3. Python2.Xwith (subprocess,commands,os,sys,git)
    Run using: 2.7.12 |Anaconda custom (64-bit)
    [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
4. NCO 4.1.0
5. GrADS/2.0.a7.oga.3


OBS datasets used if any: CERES ERBE-like ES-4 obs data source

 http://eosweb.larc.nasa.gov/PRODOCS/ceres/level3_es4_table.html
 http://eosweb.larc.nasa.gov/PRODOCS/ceres/ES4/Quality_Summaries/CER_ES4_Terra_Edition2.html

NASA CERES ERBE-like ES-4: Terra FM1 Edition2 Rev1
  2.5 Degree Regional, Monthly (Day), Total-sky and Clear-sky
  Averages

  User Applied Revision "Rev1" applied to monthly CERES Terra
  FM1 Edition2 variables for 5-year Climatology (3/00 - 2/05)
  http://eosweb.larc.nasa.gov/PRODOCS/ceres/ES4/Quality_Summaries/CER_ES4_Terra_Edition2.html#user_revision


  variables reference: Tables 4-4, 4-5 at
  http://asd-www.larc.nasa.gov/ceres/collect_guide/ES4_CG.pdf

  Referencing Data in Journal Articles:
  http://eosweb.larc.nasa.gov/PRODOCS/ceres/ES4/Quality_Summaries/CER_ES4_Terra_Edition2.html#Referencing

   Journal Article Reference:

 Wielicki, B. A., B. R. Barkstrom, E. F. Harrison, R. B. Lee III, G. L. Smith,
  and J. E. Cooper, 1996: Clouds and the Earth's Radiant Energy System (CERES):
  An Earth Observing System Experiment, Bull. Amer. Meteor. Soc., 77, 853-868.

   Acknowledgment:

  "These data were obtained from the Atmospheric Science Data Center at
   NASA Langley Research Center."

  Acknowledgment from:
  http://eosweb.larc.nasa.gov/GUIDE/dataset_documents/cer_es4.html#ack

  "These data were obtained from the NASA Langley Research Center
   Atmospheric Science Data Center."

  Datasets:
    CERES_ES4_rdir/lwcf.CER_ES4_Terra-FM1_Edition2.climo.nc
    CERES_ES4_rdir/netcf.CER_ES4_Terra-FM1_Edition2_Rev1.climo.nc
    CERES_ES4_rdir/swcf.CER_ES4_Terra-FM1_Edition2_Rev1.climo.nc
    CERES_ES4_rdir/Total-sky/net_radiant_flux.CER_ES4_Terra-FM1_Edition2_Rev1.climo.nc
    CERES_ES4_rdir/Total-sky/longwave_flux.CER_ES4_Terra-FM1_Edition2.climo.nc
    CERES_ES4_rdir/Total-sky/swabs.CER_ES4_Terra-FM1_Edition2_Rev1.climo.nc
    where CERES_ES4_rdir = /net/cjs/data/ceres/CER_ES4/Terra_Edition2/climatology/200003-200502/2.5_Degree_Regional/Monthly-Day

  have been consolidated into a compressed tar data file within this git repo which is used as
  a data source for this processor,

In [50]:
import os
os.environ['GITROOT']='/home/a1r/AM4/' #Where is your cloned git repo?
os.environ['DATAROOT']='/home/a1r/AM4/' #Where is your data repo? i.e model output and OBS that is needed from GFDL Data portal

os.environ['TMPDIR'] = "/tmp/a1r/v5" # Please provide a temporary directory.

#Note: Any reference to gcp or globus can be disregarded if run in outside of GFDL. Copy (or other relevant) can be used to replace the use of gcp elsewhere. 
os.environ['FRE_ANALYSIS_ARCHIVE']=os.environ['DATAROOT']+'/JAMES/OBS/cjs/fms/fre-analysis/'
os.environ['obs_data_file']=os.environ['FRE_ANALYSIS_ARCHIVE']+'/radiation_atmos_av_mon/'

os.environ['PATH']='/usr/local/x64/grads-2.0.a7.oga.3/Contents:/usr/local/gcp/2.3.11:/usr/local/x64/globus/5.2.1/bin:/usr/local/x64/globus/5.2.1/sbin:/usr/local/x64/netcdf-4.2_optimized/bin:/usr/local/x64/nco-4.3.1/bin:/usr/local/x64/hdf5-1.8.8_optimized/bin:/usr/local/x64/intel/Compiler/11.1/073/bin/intel64:/home/gfdl/bin:/usr/local/OSoverlay/bin:/bin:/usr/bin:/usr/local/x64/bin:/usr/local/bin:/usr/local/explorer/bin:/usr/local/pgi/linux86/bin:/usr/local/cleanscape:/usr/local/toolworks/totalview/bin:/usr/local/autodesk/maya/bin:.'
os.environ['LD_LIBRARY_PATH']='/usr/local/x64/globus/5.2.1/lib64:/usr/local/x64/nco-4.3.1/lib:/usr/local/x64/netcdf-4.2_optimized/lib:/usr/local/x64/hdf5-1.8.8_optimized/lib:/usr/local/x64/intel/Compiler/11.1/073/lib/intel64:/usr/local/globus/5.2.1/lib'
os.environ['CPATH'] ='/usr/local/x64/netcdf-4.2_optimized/include:/usr/local/x64/hdf5-1.8.8_optimized/include'
os.environ['FRE_ANALYSIS_GIT_URL']= 'file:///home/fms/local/opt/fre-analysis/git' # Please see reference in git repo
os.environ['FPATH']='/usr/local/x64/netcdf-4.2_optimized/include'
os.environ['LIBRARY_PATH']='/usr/local/x64/netcdf-4.2_optimized/lib:/usr/local/x64/hdf5-1.8.8_optimized/lib'
os.environ['GLOBUS_LOCATION']='/usr/local/x64/globus/5.2.1'
os.environ['GLOBUS_PATH']='/usr/local/x64/globus/5.2.1'
os.environ['X509_CERT_DIR']='/usr/local/x64/globus/5.2.1/etc/grid-security/certificates'
os.environ['LIBPATH']='/usr/local/x64/globus/5.2.1/lib'
os.environ['SHLIB_PATH']='/usr/local/x64/globus/5.2.1/lib'
os.environ['X509_CERT_DIR']='/usr/local/x64/globus/5.2.1/etc/grid-security/certificates'
os.environ['FRE_ANALYSIS_ARCHIVE']='/archive/fms/fre-analysis/test' # Please see reference in git repo
os.environ['_LM_FILES_']='/usr/local/Modules/modulefiles/globus/5.2.1:/usr/local/Modules/x64/modulefiles/intel_compilers/11.1.073:/usr/local/Modules/modulefiles/gcp/2.3:/usr/local/Modules/x64/modulefiles/hdf5/1.8.8:/usr/local/Modules/x64/modulefiles/netcdf/4.2:/usr/local/Modules/x64/modulefiles/nco/4.3.1:/usr/local/Modules/modulefiles/grads/2.0.a7.oga.3'
os.environ['MANPATH']='/usr/local/x64/nco-4.3.1/man:/usr/local/x64/netcdf-4.2_optimized/share/man:/usr/local/x64/intel/Compiler/11.1/073/man/en_US:/usr/local/globus/5.2.1/man:/home/gfdl/man:/usr/share/man:/usr/X11R6/man:/usr/local/x64/share/man:/usr/local/man:/usr/local/pgi/man:/usr/local/toolworks/totalview/man'
os.environ['INTEL_LICENSE_FILE']='/usr/local/x64/intel/licenses'

In [51]:
#!/usr/bin/python
############## load modules #####################
import subprocess,commands,os,sys
from subprocess import PIPE
from git import Repo

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Original Script and author:radiation_atmos_av_mon.csh ,Charles Seman (GFDL)
# Jupyter notebook development : Aparna Radhakrishnan (GFDL)
# Output: Creates figures in $out_dir/atmos_${yr1}_${yr2}
####INPUT PARAMETERS######

#Please note this script uses time averaged files as input. On the GFDL Data portal, you'll find the time-series files and a script that helps with converting them to time-averaged to work with this script. 

#For in_data_dir and in_data_file, please edit references to input from your local file system as relevant. 
in_data_dir = '/archive/Huan.Guo/awg/warsaw/c96L33_am4p0_cmip6Diag/gfdl.ncrc4-intel-prod-openmp/pp/atmos_cmip/av/monthly_35yr/'
in_data_file = 'atmos_cmip.1980-2014.{01,02,03,04,05,06,07,08,09,10,11,12}.nc'
out_dir = '/nbhome/a1r/awg/warsaw/c96L33_am4p0_cmip6Diag'
yr1 = '1980'
yr2 = '2014'
descriptor = 'AM4.0'
out_dir= '/nbhome/a1r/cjs/output/radiation_atmos_av_mon/v1110/' #Please provide an output directory
#BASEDIR="/home/a1r/ipy/Notebooks/Documentations/AM4Documentation/cjs1/" # Please see reference in git repo
BASEDIR=os.environ['GITROOT']+'/cjs1/' # Please see reference in git repo

########### end of analysis template variables  #######
platform = 'desktop' 

In [52]:
####### load modules ###########
TMPDIR = os.environ["TMPDIR"]
FRE_CODE_BASE = BASEDIR
PACKAGE_NAME = 'radiation_atmos_av_mon'
if not os.path.exists(TMPDIR):
     os.makedirs(TMPDIR)
if not os.path.exists(out_dir):
     os.makedirs(out_dir)            
if ( platform == 'hpcs-csc' ) or ( platform == 'desktop'):
    print("Running script on non-GFDL machine and public repo: skip module load") 
else:
    print "ERROR: invalid platform: %s" % platform
    exit(1)

#---- Check if environment loaded correctly ---#

###################
# run the script
##################


options = "-i "+in_data_dir+" -d "+descriptor+" -y "+yr1+","+yr2+" -o "+out_dir

command = FRE_CODE_BASE+"/"+PACKAGE_NAME+"/radiation_atmos_av_mon.csh "+options+" "+in_data_file

print(command)

cmd = subprocess.Popen('tcsh -c "'+command+'"',shell=True,stdout=PIPE,stdin=PIPE, stderr=PIPE)
output, errors = cmd.communicate()
print output, errors 

Running script on non-GFDL machine and public repo: skip module load
/home/a1r/AM4//cjs1//radiation_atmos_av_mon/radiation_atmos_av_mon.csh -i /archive/Huan.Guo/awg/warsaw/c96L33_am4p0_cmip6Diag/gfdl.ncrc4-intel-prod-openmp/pp/atmos_cmip/av/monthly_35yr/ -d AM4.0 -y 1980,2014 -o /nbhome/a1r/cjs/output/radiation_atmos_av_mon/v1110/ atmos_cmip.1980-2014.{01,02,03,04,05,06,07,08,09,10,11,12}.nc

Linux a1r 2.6.32-696.13.2.el6.x86_64 #1 SMP Fri Sep 22 12:32:14 EDT 2017 x86_64 x86_64 x86_64 GNU/Linux

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
/home/a1r/AM4//cjs1//radiation_atmos_av_mon/radiation_atmos_av_mon.csh -i /archive/Huan.Guo/awg/warsaw/c96L33_am4p0_cmip6Diag/gfdl.ncrc4-intel-prod-openmp/pp/atmos_cmip/av/monthly_35yr/ -d AM4.0 -y 1980,2014 -o /nbhome/a1r/cjs/output/radiation_atmos_av_mon/v1110/ atmos_cmip.1980-2014.01.nc atmos_cmip.1980-2014.02.nc atmos_cmip.1980-2014.03.nc atmos_cmip.1980-2014.04.nc atmos_cmip.1980-2014.05.nc atmos_cmip.1980-2014.06.nc atmos_c